In [1]:

import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import random
import sys
import os


Using TensorFlow backend.


In [2]:
filename = "moes_tavern_lines.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [3]:
print('corpus length:', len(raw_text))

corpus length: 305270


In [4]:
chars_new=set(raw_text)
words=set(raw_text.split())
print(chars_new)
print(words)


{'è', ';', '%', 'z', ')', 'à', '#', 'o', 'w', 'v', ' ', 'y', "'", 'j', '$', 's', 'r', '(', 'n', '9', 'm', 'i', '-', '/', '6', '[', 'ä', 'a', 'b', '0', 'ã', 'é', '_', 'd', '©', '!', 'h', '"', 'x', 'u', '.', '7', '3', '8', '?', '\n', 'p', '2', 'q', 'c', 'l', ']', ',', '5', '4', 'f', ':', 'ó', 't', 'k', '1', 'e', 'ü', '&', 'g'}
{'bucks?', 'shindig', 'ohmygod,', '(strangles', 'stories,', 'white.', 'tale', 'penny!', 'whisper', 'secrets', 'voters', 'november.', 'warren', "treatin'", 'finger)', '(stupidly)', 'that.', 'eliminate', 'byrne?!', 'thoughtless', 'farewell...', 'i-i-i', 'buttons.', 'sen...', 'hiya', 'humiliation', 'designated', 'radiation', 'wears', 'guide', 'another', 'everyday!', 'email', 'glen:', 'ohh...', 'outside.', 'theater', 'water?', '(furiously)', 'oopsie.', 'we', 'dear?', 'bon', '(super', "y'see?", 'carl', 'long', 'layer', 'carl,', '(sobs', 'touchdown!', 'dad,', 'resenting', 'faced.', 'public', 'game', 'odd', 'manipulation', 'camp,', 'eye?!', 'restaurants', 'drinking:', 're

In [7]:
print(type(chars_new))
print(type(words))


<class 'set'>
<class 'set'>


In [8]:
##converting word to indexes
word_indices = dict((c, i) for i, c in enumerate(words))

## converting indexes to word
indices_word = dict((i, c) for i, c in enumerate(words))

In [9]:

print(type(word_indices), "length:",len(word_indices) )
print(type(indices_word), "length", len(indices_word))

<class 'dict'> length: 10353
<class 'dict'> length 10353


In [10]:
maxlen = 30
step = 3
sentences = []
next_words = []
next_words= []
sentences1 = []
list_words = []

sentences2=[]
list_words=raw_text.lower().split()

In [11]:
for i in range(0,len(list_words)-maxlen, step):
    sentences2 = ' '.join(list_words[i: i + maxlen])
    sentences.append(sentences2)
    next_words.append((list_words[i + maxlen]))
print('nb sequences(length of sentences):', len(sentences))
print("length of next_word",len(next_words))

nb sequences(length of sentences): 16319
length of next_word 16319


In [12]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence.split()):
        #print(i,t,word)
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1


Vectorization...


In [13]:
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(words))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
#model.add(Dense(1000))
model.add(Activation('softmax'))

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [15]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))


In [16]:
for iteration in range(1, 300):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=2)
    model.save_weights('GoTweights',overwrite=True)

    start_index = random.randint(0, len(list_words) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)
        generated = ''
        sentence = list_words[start_index: start_index + maxlen]
        generated += ' '.join(sentence)
        print('----- Generating with seed: "' , sentence , '"')
        print()
        sys.stdout.write(generated)
        print()

        for i in range(1024):
            x = np.zeros((1, maxlen, len(words)))
            for t, word in enumerate(sentence):
                x[0, t, word_indices[word]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            generated += next_word
            del sentence[0]
            sentence.append(next_word)
            sys.stdout.write(' ')
            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1


/usr/local/lib/python3.5/dist-packages/Keras-2.1.3-py3.5.egg/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/2
16319/16319 [==============================] - 80s 5ms/step - loss: 7.5991
Epoch 2/2
16319/16319 [==============================] - 74s 5ms/step - loss: 7.2707

----- diversity: 0.2
----- Generating with seed: " ['it', 'coming.', '(shaking', 'head)', "that's", 'how', 'it', 'is,', 'though:', 'one', 'day', 'your', 'wife', 'is', 'making', 'you', 'your', 'favorite', 'meal,', 'the', 'next', 'day', "you're", 'thawing', 'a', 'hot', 'dog', 'in', 'a', 'gas'] "

it coming. (shaking head) that's how it is, though: one day your wife is making you your favorite meal, the next day you're thawing a hot dog in a gas
 you moe_szyslak: moe_szyslak:

ValueError: sum(pvals[:-1]) > 1.0

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  305270
Total Vocab:  65


In [6]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  305170


In [8]:

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [11]:
model.fit(X, y, epochs=1, batch_size=128, callbacks=callbacks_list)

Epoch 1/1
305152/305170 [============================>.] - ETA: 0s - loss: 2.9565
Epoch 00001: loss improved from inf to 2.95652, saving model to weights-improvement-01-2.9565.hdf5
305170/305170 [==============================] - 976s 3ms/step - loss: 2.9565


In [12]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [16]:
import sys
filename = "weights-improvement-01-2.9565.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")


Seed:
" ooh, duff luck.
carl_carlson: i never woulda figured that out.
lenny_leonard: hm. now, that's the ki "
 toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe 